# YouTube API Connector - Country/Day/Video Granularity

First of all, let's import the libraries to connect to the YouTube API and manipulate the data:

In [1]:
# Importing library
import os
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import googleapiclient.errors

Then, let's add the credentials to connect to the YouTube Data v3 API. This will provide us a generic information about the channel, including the ID of the playlist where all the videos are updated.

In [2]:
# Defining the API Key
api_key = 'api_key'

# Defining the API name and version before connection
youtube_api_service_name = "youtube"
youtube_api_version = "v3"

youtube = build(youtube_api_service_name, youtube_api_version, developerKey = api_key)

# Gathering the channel content details
request = youtube.channels().list(part='ContentDetails', id = 'UCzQTrA_c1BgRNLewVyt2UFw')

response = request.execute()

print(response)

{'kind': 'youtube#channelListResponse', 'etag': 'xm0qCYcUUWM0F3XMlRFDWwej_n8', 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5}, 'items': [{'kind': 'youtube#channel', 'etag': 'eyLQNm6fwyqFzcXjoobAzVEH-Cw', 'id': 'UCzQTrA_c1BgRNLewVyt2UFw', 'contentDetails': {'relatedPlaylists': {'likes': '', 'uploads': 'UUzQTrA_c1BgRNLewVyt2UFw'}}}]}


In the next step, this playlist ID will allow to provide us to get all the video IDs, the title, the description, and the publish date. I have created a function to gather all this data for this step.

As it is only possible to get the data from only 50 videos, it is necessary to use the 'nextPageToken' to extract the data from the rest of the videos.

In [3]:
# Adding read permissions to the API
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

# Creating a function to gather all the video information from the previous playlistId
def gather_youtube_videos(playlistId):

    youtube = build(youtube_api_service_name, youtube_api_version, developerKey = api_key)
    res = youtube.playlistItems().list(part="snippet", playlistId='UUzQTrA_c1BgRNLewVyt2UFw', maxResults="50").execute()

    nextPageToken = res.get('nextPageToken')
    
    # Creating a while structure to gather the video from the rest of the pages. 
    #It will stop when there hare no more 'nextPageToken' available
    while ('nextPageToken' in res):
        nextPage = youtube.playlistItems().list(
        part="snippet",
        playlistId=playlistId,
        maxResults="50",
        pageToken=nextPageToken
        ).execute()
        
        res['items'] = res['items'] + nextPage['items']

        if 'nextPageToken' not in nextPage:
            res.pop('nextPageToken', None)
        else:
            nextPageToken = nextPage['nextPageToken']

    return res

In [4]:
# Apply the function over the PlayListId to gather the basic data from the videos
info_videos = gather_youtube_videos('UUzQTrA_c1BgRNLewVyt2UFw')

Let's extract the basic information for each video from the previous dictionary:

In [5]:
# Defining the variables to store the data from the dictionary
video_id = []

# Gather all the video IDs from the dictionary
for item in info_videos["items"]:
    video_id.append(item['snippet']['resourceId']['videoId'])

In the following step, I will gather different metrics (views, likes, shares...) with a daily granularity per video and country:

In [6]:
# Defining the list of countries
country = ['ES','MX','AR','BO','UY','PE','CL','CO','US']

In [8]:
# Creating a dictionary to store the data
dic = {}

# Creating a function to gather the data
def main():
    
    scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

    api_service_name = "youtubeAnalytics"
    api_version = "v2"
    client_secrets_file = "file_name.json"

    # Get credentials and create API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube_analytics = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)

    # Get the metrics for each video
    for i in range(len(video_id)):
        for j in range(len(country)):
            request1 = youtube_analytics.reports().query(
            dimensions="day",
            endDate="2022-10-08",
            ids="channel==MINE",
            maxResults=150,
            metrics="views,likes,comments,dislikes,shares,subscribersLost,subscribersGained,estimatedMinutesWatched,averageViewDuration,averageViewPercentage,annotationImpressions,annotationClicks,annotationClickThroughRate,cardImpressions,cardClicks,cardClickRate",
            filters = f'video=={video_id[i]};country=={country[j]}',
            startDate="2021-03-08"
    )
            response2 = request1.execute()
        
            dic[video_id[i],country[j]] = response2['rows']
        

if __name__ == "__main__":
    main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=852632511285-09e8025vv77tqnorhjgp8lof8j6j67cm.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=MlV19O4JiFpb7RpzmO8E0qKBNAtyf9&prompt=consent&access_type=offline
Enter the authorization code: 4/1ARtbsJqHE7TGR-UYe4NprMrjSeNR43uKs3lS6LSxY9K49YHlIlqvrONjI-I


Now, all the data gathered has been stored in a dictionary. Let's transform that dictionary into a DataFrame.

In [9]:
# Create a list with the variables extracted from each video
video_metrics = ['date', 'views','likes','comments','dislikes','shares','subscribersLost','subscribersGained',
           'estimatedMinutesWatched','averageViewDuration','averageViewPercentage','annotationImpressions',
           'annotationClicks','annotationClickThroughRate','cardImpressions','cardClicks','cardClickRate']

# Use the .items() method to store the key and the values separately in the DataFrame
df_metrics = pd.DataFrame(dic.items())
df_metrics

# Renaming the columns
df_metrics.rename(columns={0: 'video_id', 1: 'metrics'}, inplace=True, errors='raise')

I will use the function .explode() in the next step to transform each element of a list-like to a row, replicating the index values. That's why I will a reset_index() function at the end of the chain.

In [10]:
# Use .explode() function to desaggregate the lists on the metrics columns
df_stacked = df_metrics.explode('metrics').reset_index(drop=True)

Now, let's do some final transformations before exporting this DataFrame to .csv

In [11]:
# Changing the metrics variable as string to do some replacements
df_stacked['video_id'] = df_stacked['video_id'].astype(str)

# Replace the brackers by nothing
df_stacked['video_id'] = df_stacked['video_id'].str.replace('(', '')
df_stacked['video_id'] = df_stacked['video_id'].str.replace(')', '')

# Changing the metrics variable as string to do some replacements
df_stacked['metrics'] = df_stacked['metrics'].astype(str)

# Replace the brackers by nothing
df_stacked['metrics'] = df_stacked['metrics'].str.replace('[', '')
df_stacked['metrics'] = df_stacked['metrics'].str.replace(']', '')

C:\Users\xsof\AppData\Local\Temp\ipykernel_1984\2115065348.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_stacked['video_id'] = df_stacked['video_id'].str.replace('(', '')
C:\Users\xsof\AppData\Local\Temp\ipykernel_1984\2115065348.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_stacked['video_id'] = df_stacked['video_id'].str.replace(')', '')
C:\Users\xsof\AppData\Local\Temp\ipykernel_1984\2115065348.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_stacked['metrics'] = df_stacked['metrics'].str.repl

In [12]:
# Split the 'video_id' column into several columns
df_stacked[['video_id2','country']] = df_stacked['video_id'].str.split(',', expand = True)

# Changing 'country' column data type
df_stacked['country'] = df_stacked['country'].astype(str)

# Dropping and renaming the column video_id
df_stacked.drop('video_id', axis = 1, inplace = True)
df_stacked.rename(columns = {'video_id2':'video_id'}, inplace = True)
df_stacked = df_stacked[df_stacked['metrics'].notna()]

In [13]:
# Split the 'Metrics' columns into several columns
df_stacked[video_metrics] = df_stacked['metrics'].str.split(',', expand = True)

#Change Data type to remove the quotations marks correctly on the 'Date' variable
df_stacked['date'] = df_stacked['date'].astype(str)

# Replacing quotation marks by nothing
df_stacked['video_id'] = df_stacked['video_id'].str.replace("'", '')
df_stacked['country'] = df_stacked['country'].str.replace("'", '')
df_stacked['date'] = df_stacked['date'].str.replace("'", '')

In [14]:
# Removing the old 'Metrics' variable
df_stacked.drop('metrics', axis = 1, inplace = True)

# Creating the CSV file
df_stacked.to_csv('video_performance_country.csv', index = False)

In the following steps, I will connect to Google BigQuery with two different goals:

1. Create a table to store the data recently extracted
2. Pull the output file got in the previous step to that Google BigQuery table

In [15]:
# Importing libraries to connect with Google BigQuery
from google.cloud import bigquery
from google.oauth2 import service_account

# Adding the credentials from the JSON file to connect with the platform
credentials = service_account.Credentials.from_service_account_file(
'misuperproyecto-06012a0c4865.json')
project_id = 'misuperproyecto'
client = bigquery.Client(credentials= credentials, project=project_id)

In [16]:
# Set table_id to the ID of the table to create
table_id = 'misuperproyecto.youtube_channel_data.fact_video_country'

# Add the schema of the table
schema = [
    bigquery.SchemaField("video_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("country", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("date", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("views", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("likes", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("comments", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("dislikes", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("shares", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("subscribersLost", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("subscribersGained", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("estimatedMinutesWatched", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("averageViewDuration", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("averageViewPercentage", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("annotationImpressions", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("annotationClicks", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("annotationClickThroughRate", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("cardImpressions", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("cardClicks", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("cardClickRate", "INTEGER", mode="NULLABLE"),
]

# Make an API request to create the table
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)  
print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))

Created table misuperproyecto.youtube_channel_data.fact_video_country


Finally, let's push the .csv() file created to Google BigQuery

In [17]:
# Specify the name of the file
file_path = 'video_performance_country.csv'

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1, autodetect=True,
)

# Open and read the file
with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)
job.result() 

# Make an API request to upload the table
table = client.get_table(table_id)  
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 2889 rows and 19 columns to misuperproyecto.youtube_channel_data.fact_video_country
